In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
import gzip

import numpy as np
from IPython import display

import time

from keras import backend as K
from keras.callbacks import Callback
from keras.models import Model, load_model, model_from_json
from keras.optimizers import SGD, Adam
from keras.utils import HDF5Matrix
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Flatten, Concatenate 
# Conv2D, Input, Flatten, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras.applications.vgg19 import VGG19

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

In [9]:
import sys
sys.path.append("..")

In [10]:
from utils.load_data import load_data
from utils.preprocess import DataGenerator
from utils.comparams import calculate_auc, auc
from utils.callbacks import PlotCurves

In [11]:
def build_arch_2_net():
    
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = vgg19.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    y = Dense(2, activation='softmax')(x) # sigmoid instead of softmax to have independent probabilities
    model = Model(inputs=vgg19.input, outputs=y)
    
    # Train only the top layer
    for layer in vgg19.layers:
        layer.trainable = False

    return model

def build_arch_4_net():
    
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = vgg19.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    y = Dense(2, activation='softmax')(x) # sigmoid instead of softmax to have independent probabilities
    model = Model(inputs=vgg19.input, outputs=y)
    
    # Train only the top layer
    for layer in vgg19.layers:
        layer.trainable = False

    return model

def build_arch_5_net():
    
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = vgg19.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    y = Dense(2, activation='softmax')(x)
    model = Model(inputs=vgg19.input, outputs=y)
    
    # Train only the top layer
    for layer in vgg19.layers:
        layer.trainable = False

    return model

def build_arch_6_net():
    
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = vgg19.output
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    y = Dense(2, activation='softmax')(out) # sigmoid instead of softmax to have independent probabilities

    model = Model(inputs=vgg19.input, outputs=y)
    
    # Train only the top layer
    for layer in vgg19.layers:
        layer.trainable = False

    return model


def build_arch_8_net():
    
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = vgg19.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    y = Dense(2, activation='softmax')(x)
    model = Model(inputs=vgg19.input, outputs=y)
    
    # Train only the top layer
    for layer in vgg19.layers:
        layer.trainable = False

    return model

# DATA

In [13]:
data_dir = '../data'

x_test, y_test_true = load_data(data_dir, purpose='test')#, norm='macenko')

# indexes
test_id = np.arange(len(x_test))

# create a useful dictionary structures
partition = {}
partition['test'] = test_id
    
test_labels = {str(i) : y_test_true[i].flatten()[0] for i in test_id}

len(test_labels)

32768

## MODEL

In [14]:
file_dir = '../Model'
model_name = 'vgg19_model_bin_cross_arch_8_lr_0_0001'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

dependencies = {
    'auc': auc
}

In [ ]:
model = load_model(network_filepath, custom_objects=dependencies)

preds = model.predict_generator(test_generator)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.


In [ ]:
##### Calculate AUC 
true_labels = np.array(y_test_true).flatten()
pred_labels = np.array([p[1] for p in preds])
calculate_auc(true_labels, pred_labels)

#### BEST ACC MODEL

In [ ]:
file_dir = '../Model/weights'
model_name = 'vgg19_model_bin_cross_arch_8_lr_0_0001best_acc_model'
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

dependencies = {
    'auc': auc
}

In [ ]:
model = build_arch_8_net()

In [ ]:
model.load_weights(network_filepath)

preds = model.predict_generator(test_generator)

In [ ]:
# whole data normal data predictions inception resnet
true_labels = np.array(y_test_true).flatten()[:len(preds)]

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels) # without beta lr=0.0001

#### BEST AUC MODEL

In [ ]:
file_dir = '../Model/weights'
model_name = 'vgg19_model_bin_cross_arch_8_lr_0_0001best_auc_model'
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

dependencies = {
    'auc': auc
}

In [ ]:
model = build_arch_8_net()

In [ ]:
model.load_weights(network_filepath)

preds = model.predict_generator(test_generator)

In [ ]:
# whole data normal data predictions inception resnet
true_labels = np.array(y_test_true).flatten()[:len(preds)]

pred_labels = np.array([p[1] for p in preds])

calculate_auc(true_labels, pred_labels) # without beta lr=0.0001